In [10]:
pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 251 kB 807 kB/s eta 0:00:01
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [13]:
from dash import Dash, html, dash_table, dcc
import plotly.express as px
import pandas as pd

df = pd.read_excel('egrid2022_data.xlsx', sheet_name='BA22')

app = Dash()

app.layout = [
    html.Div(children='My First App with Data'),
    dash_table.DataTable(data=df.to_dict('records'), page_size=10),
    dcc.Graph(figure=px.histogram(df, x='Balancing Authority Name', y='BA annual CO2 equivalent total output emission rate (lb/MWh)', histfunc='avg'))
]

#Need to run the app - do not change! 
if __name__ == '__main__':
    app.run(debug=True)

In [8]:
import pandas as pd
import numpy as np
import geopandas as gp

#Plots which NERC Coordinator is responsible for which part of the US
nercCoordinators = gp.read_file("/home/jupyter/cloud-ai/NERC_Reliability_Coordinators.shp")

nercCoordinators.plot(column = "NAME", 
               cmap = "gist_ncar", 
               legend = True, 
               legend_kwds={"loc": "center left", "bbox_to_anchor": (1, 0.5)},
              ).set_axis_off()

#Plots the peak load of each energy planning area in the US
planningAreas = gp.read_file("/home/jupyter/cloud-ai/Planning_Areas.shp")

planningAreas.plot(column = "PEAK_LOAD",
                   cmap = "Reds",
                   legend = True,
                   legend_kwds={'shrink': 0.4, "label": "Peak Load"}
                  ).set_axis_off()

DriverError: /home/jupyter/cloud-ai/NERC_Reliability_Coordinators.shp: No such file or directory